In [38]:
import pyrosetta
import pyrosetta.distributed.dask
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score
from dask.distributed import Client, LocalCluster



In [43]:
my_flags = "-extra_res_fa ATP.am1-bcc.fa.params 295.am1-bcc.fa.params -score:extra_torsion_file Temp_ATP/ATP.am1-bcc.fa.tors"

pyrosetta.distributed.dask.init_notebook(my_flags)

In [44]:
pose_start = score.ScorePoseTask(weights="ref2015_cart")(io.pose_from_file("CDK9_ATP_3BLQ_relaxed.pdb"))

core.import_pose.import_pose: File 'CDK9_ATP_3BLQ_relaxed.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 14 atoms at position 296 in file CDK9_ATP_3BLQ_relaxed.pdb. Best match rsd_type:  ATP
core.scoring.CartesianBondedEnergy: Creating new peptide-bonded energy container (296)


In [41]:
def run_fast_design(pose_start):
    
    pose_start = packed_pose.to_pose(pose_start)
    
    scorefxn_cart = pyrosetta.create_score_function("ref2015_cart")
    atp_res = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("X")
    
    # Selector for repacking NOT ligand + small shell
    neighboor = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector=atp_res, 
                                                                                           distance=8,
                                                                                           include_focus_in_subset=False)
    
    # Selector for minimization Ligand + broad shell
    neighboor2 = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector=atp_res, 
                                                                                           distance=16,
                                                                                           include_focus_in_subset=True)
    
    not_neighboor = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(neighboor)
    not_neighboor2 = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(neighboor2)

    # Initiate TaskFactory
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())

    # Disable design outside design shell
    tf.push_back(
        pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
            pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT(), not_neighboor)
    )

    
    # Remove Cys from design
    aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
    aa_to_design.aas_to_keep("ADEFGHIKLMNPQRSTVWY")
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(aa_to_design, neighboor))
                 
                 
    # Convert task factory into PackerTask just to view 
    #packer_task = tf.create_task_and_apply_taskoperations(pose_start)
    
    
    # Set MoveMapFactory
    
    # Set up a MoveMapFactory
    mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
    mmf.all_bb(setting=True)  # Set to true if needed
    mmf.all_bondangles(setting=True)
    mmf.all_bondlengths(setting=True)
    mmf.all_chi(setting=True)
    mmf.all_jumps(setting=True)
    mmf.set_cartesian(setting=True)

# Set movemap actions to turn on or off certain torsions, overriding the above defaults
    enable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_enable
    disable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_disable

# Set custom minimizable torsions

    mmf.add_bondangles_action(action=disable, selector=neighboor2)
    mmf.add_bondlengths_action(action=disable, selector=neighboor2)
    mmf.add_chi_action(action=disable, selector=neighboor2)

    
    
    

    # Set up FastDesign

    fast_design = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn, 
                                                                              standard_repeats=5)
    fast_design.set_task_factory(tf)
    fast_design.set_movemap_factory(mmf)
    fast_design.set_scorefxn(scorefxn_cart)
    fast_design.set_cartesian(True)
    
    # Apply FastDesign
    fast_design.apply(pose_start)
    
    return score.ScorePoseTask(weights="ref2015_cart")(packed_pose.to_packed(pose_start))



In [3]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)
 
client.run(pyrosetta.distributed.dask.init_notebook, my_flags)

futures = [client.submit(run_fast_design, pose_start) for i in range(100)]
logging.info("Now performing refinement on each point mutant!")
results = [future.result() for future in futures]

client.close()
cluster.close()

core.import_pose.import_pose: File 'CDK9_ATP_3BLQ_relaxed.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 14 atoms at position 296 in file CDK9_ATP_3BLQ_relaxed.pdb. Best match rsd_type:  ATP
